<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Fall 2021  </h1>
<h1 style='text-aling:center;color:Navy'>  Assignment 7 </h1>

This assignment is focused on Time Series Analysis and Big Graph Processing with GraphLab.

To complete the assignment, you should complete this notebook by filling in the cells provided.

<b>Submission Deadline: This assignment is due Wednesday, Nov 28 at 11:59 P.M</b>

A few notes before you start:
- Directly sharing answers is not okay, but discussing problems with other students is encouraged.
- You should start early so that you have time to get help if you're stuck.

- Before continuing the assignment, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to okpy.org and flag the correct version. There will be another submit cell at the end of the assignment when you finish!

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>
<br>
Before you begin completing the assignment, execute the following cell to load the provided tests.

In [34]:
# Don't change this cell; just run it. 
# When you log-in please hit return (not shift + return) after typing in your email
from client.api.notebook import Notebook
ok = Notebook('Assignment7.ok')

Assignment: Assignment7: Time Series Analysis and Big Graph Processing with GraphLab
OK, version v1.15.44



In [35]:
_ = ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... 

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
ERROR  | auth.py:101 | {'error': 'invalid_grant'}


Saved 'Assignment7.ipynb'.
Performing authentication
Please enter your bCourses email: vijayasimha.bheemireddy@ucdenver.edu
Unable to find an open port for authentication.

Copy the following URL and open it in a web browser. To copy,
highlight the URL, right-click, and select "Copy".

https://kubemaster.ucdenver.pvt:5000/client/login/

After logging in, copy the code from the web page, paste it below,
and press Enter. To paste, right-click and select "Paste".

Paste your code here: uZCDkjJXAzUEe284HKsSseDfITdRNn


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Successfully logged in as vijayasimha.bheemireddy@ucdenver.edu
Submit... 100% complete
Submission successful for user: vijayasimha.bheemireddy@ucdenver.edu
URL: https://kubemaster.ucdenver.pvt:5000/UCDenver/CSCI4951-5951/fa21/assignment7/submissions/b8qvjd



<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style="color:#3665af">SECTION 1: Time Series Analysis </span>
<hr>

###### Goal
This assignment covers the topic of time series analysis using TimescaleDB. Make sure you have
your system fully configured before starting this assignment. Refer to the Assignment document to do so.

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Connect to PostgreSQL</div>

This section includes a reminder of how to connect to PostgreSQL.<br>
Open PostgreSQL and open the nyc_data database:

```
psql -U postgres -h localhost -d nyc_data
```

Then use the following command to see the list of tables:

```
\dt
```

<img src="img/img1.png" style="width:800px;">

Now use the following command to see the columns of rides table (you can press q for going back to the terminal):
```
\d rides
```

<img src="img/img2.png" style="width:600px;">

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">Run and Study Query Examples</div>

## Example 1

Find the average fare of rides with 2+ passengers per day until '2016-01-08':

```
SELECT date_trunc('day', pickup_datetime) as day, avg(fare_amount)
FROM rides
WHERE passenger_count > 1 AND pickup_datetime < '2016-01-08'
GROUP BY day
ORDER BY day;
```

<img src="img/img3.png" style="width:600px;">

## Example 2
Total number of rides by day for first 5 days

```
SELECT date_trunc('day', pickup_datetime) as day, COUNT(*) FROM rides
GROUP BY day
ORDER BY day
LIMIT 5;
```

<img src="img/img4.png" style="width:600px;">

Note: The following query is not supported by PostgreSQL itself and time_bucket is provided by timescaledb.

## Example 3
Find the number of rides by 5-minute intervals on 2016-01-01 using the TimescaleDB "<i>time_bucket</i>" function

```
SELECT time_bucket('5 minute', pickup_datetime) as five_min, count(*)
FROM rides
WHERE pickup_datetime < '2016-01-01 02:00'
GROUP BY five_min
ORDER BY five_min;
```

<img src="img/img5.png" style="width:600px;">

## Example 4
Analyze rides by rate type:

```
SELECT rate_code, COUNT(vendor_id) as num_trips FROM rides
WHERE pickup_datetime < '2016-01-08'
GROUP BY rate_code
ORDER BY rate_code;
```

<img src="img/img6.png" style="width:600px;">

<i>rate_code</i> doesn't really tell us what these groups represent, and it doesn't look like there is any other
info on rates in the rides table. However, there is a separate rates table and TimescaleDB supports JOINs
between tables:

```
SELECT rates.description, COUNT(vendor_id) as num_trips
FROM rides JOIN rates on rides.rate_code = rates.rate_code
WHERE pickup_datetime < '2016-01-08'
GROUP BY rates.description ORDER BY rates.description;
```

<img src="img/img7.png" style="width:600px;">

Now we have something that is human readable. In particular, two of these rate types correspond to local airports (JFK, Newark). Let's take a closer look at those two:

```
SELECT rates.description, COUNT(vendor_id) as num_trips,
        AVG(dropoff_datetime - pickup_datetime) as avg_trip_duration,
        AVG(total_amount) as avg_total,
        AVG(tip_amount) as avg_tip, MIN(trip_distance) as min_distance,
        AVG(trip_distance) as avg_distance, MAX(trip_distance) as
        max_distance,
        AVG(passenger_count) as avg_passengers
FROM rides JOIN rates on rides.rate_code = rates.rate_code
WHERE rides.rate_code in (2,3) AND pickup_datetime < '2016-02-01'
GROUP BY rates.description
ORDER BY rates.description;
```

<div style="font-size:30px;color:#3665af;background-color:#e1dfb1;padding:10px;">Exercise - Write Queries</div>

<div style="font-size:20px;color:#F1F8FC;background-color:#557aba;padding:10px;">Query 1</div>


Write a query to find the total number of rides, total distance, and total fare for each day and rate_code. Order the result based on day and then rate_code. Do you see any unexpected number(s) in the result suggesting inaccuracy in the dataset?

Include your query and its result (copy and paste) into the text file. Do this for Q2 and Q3 as well. Output format should be as follows:

<img src="img/img8.png" style="width:600px;">

<i>Note that this result is computed over a different dataset. Your result would be different but should follow
the same format</i>

<div style="font-size:20px;color:#F1F8FC;background-color:#557aba;padding:10px;">Query 2</div>

Find the change in the number of rides for each rate_code/hour between 2016-01-01 and 2016-01-02.

For example, if there are 10 rides for rate_code <font color='blue'>99</font> between <font color='orange'>2pm</font> and 3pm on 2016-01-01 and there are 5 rides for the same time and rate_code on 2016-01-02, the result must contain <font color='orange'>2</font>, <font color='blue'>99</font>, -5.


You might find the following functions useful:
- extract(field from timestamp) extracts hour from date
    - extract(hour from timestamp '2001-02-16 20:38:40') returns 20
- date(datetime) extracts date from datetime
    - date('2001-02-16 20:38:40') = '2001-02-16'

<div style="font-size:20px;color:#F1F8FC;background-color:#557aba;padding:10px;">Query 3</div>

Write a query to find the most expensive <i>rate_code (i.e., the highest avg(fare_amount))</i> for each hour of the day on 2016-01-01. The result should contain the date and hour (e.g., 2016-01-01 01:00:00), rate_code, and the average fare for that rate_code. Please note that <b>only</b> the most expensive rate code for each hour must be included in the result. For example, if rate_code 1 is the most expensive between 1pm and 2pm, other rate_codes must not appear in the result for 1-2pm.

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

<br>

# <span style="color:#3665af">SECTION 2: Big Graph Processing with GraphLab </span>
<hr>

###### Goal
In this assignment, we will learn how to work with a graph-based distributed computation framework to read graph data and calculate graph properties including triangle counts connected components and PageRank.



In [1]:
#Please run this command to install turicreate
!pip install turicreate

     |████████████████████████████████| 92.0 MB 7.5 kB/s  eta 0:00:01
     |████████████████████████████████| 3.6 MB 99.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 68.9 MB/s eta 0:00:01
     |████████████████████████████████| 86.4 MB 30 kB/s /s eta 0:00:01
     |████████████████████████████████| 322 kB 26.3 MB/s eta 0:00:01
     |████████████████████████████████| 18.3 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 89.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 274 kB/s  eta 0:00:01
     |████████████████████████████████| 449 kB 83.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 90.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 158 kB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 77.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 98.3 MB/s eta 0:00:01
     |███████████████████████████

In [2]:
import turicreate

To facilitate graph-oriented data analysis, GraphLab offers an SGraph object, a scalable graph data structure backed by SFrames (https://turi.com/products/create/docs/generated/graphlab.SGraph.html).
You will need to briefly read through https://apple.github.io/turicreate/docs/api/ to apply different functions for Section2



First we create a graph. We can start with empty graph, then add vertices and edges in the form of lists of GraphLab.

In [3]:
from turicreate import SGraph, Vertex, Edge
g = SGraph()
verts = [Vertex(0, attr={'breed': 'labrador'}),
         Vertex(1, attr={'breed': 'Siberian Husky'}),
         Vertex(2, attr={'breed': 'vizsla'})]
g = g.add_vertices(verts)
g = g.add_edges(Edge(1, 2))
print(g)

SGraph({'num_edges': 1, 'num_vertices': 3})


In [4]:
#Inspecting SGraphs
print(g.summary())

{'num_edges': 1, 'num_vertices': 3}


Another way to create a graph is from an edge list stored in an SFrame
https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html?highlight=sframe

In [8]:
from turicreate import SFrame
edge_data = SFrame.read_csv('/home/jovyan/bigdata/Assignment7/data/star-wars-network-edges.csv')

g = SGraph()
g = g.add_edges(edge_data, src_field='source', dst_field='target')
print(g)

Finished parsing file /home/jovyan/bigdata/Assignment7/data/star-wars-network-edges.csv

------------------------------------------------------

Parsing completed. Parsed 60 lines in 0.036113 secs.


Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/jovyan/bigdata/Assignment7/data/star-wars-network-edges.csv

Parsing completed. Parsed 60 lines in 0.003881 secs.

SGraph({'num_edges': 60, 'num_vertices': 21})


Here we show how to save the SGraph in binary format to a new folder called "star-wars-network-edges" and re-load it with a different name:

In [14]:
import turicreate as gl
g.save('star-wars-network-edges')
new_graph = gl.load_sgraph('star-wars-network-edges')

***
<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 1
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    How many vertices and edges are in the graph? 
    </div>
</div>

<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 2
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Which vertex has the highest degree? (highest number of edges).
    </div>
</div>

***
For big graphs, we can print summary of statistics:

In [25]:
deg.summary()

Class                                    : DegreeCountingModel

Graph
-----
num_edges                                : 367662
num_vertices                             : 36692

Results
-------
graph                                    : SGraph. See m['graph']

Metrics
-------
training time (secs)                     : 0.3003

Queryable Fields
----------------
component_id                             : An SFrame with each vertex's component id
component_size                           : An SFrame with the size of each component
graph                                    : A new SGraph with the color id as a vertex property
training_time                            : Total training time of the model



In [ ]:
## Place your answer Here

Class                                    : DegreeCountingModel

Graph
-----
num_edges                                : 367662
num_vertices                             : 36692

Results
-------
graph                                    : SGraph. See m['graph']

Metrics
-------
training time (secs)                     : 0.3003

Queryable Fields
----------------
component_id                             : An SFrame with each vertex's component id
component_size                           : An SFrame with the size of each component
graph                                    : A new SGraph with the color id as a vertex property
training_time                            : Total training time of the model


# Load graph data : Enron email network (edge indicated that email was exchanged, undirected edges)

In [10]:
g = gl.load_sgraph('http://snap.stanford.edu/data/email-Enron.txt.gz', 'snap')

Downloading http://snap.stanford.edu/data/email-Enron.txt.gz to /var/tmp/turicreate-jovyan/186/8b682e9c-aa69-4fba-b505-bd494c7d6c2f.gz

Finished parsing file http://snap.stanford.edu/data/email-Enron.txt.gz

Parsing completed. Parsed 367662 lines in 0.071031 secs.

***
<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 3
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Report number of vertices and edges.
    </div>
</div>

In [27]:
g = gl.load_sgraph('http://snap.stanford.edu/data/email-Enron.txt.gz','snap')

Finished parsing file http://snap.stanford.edu/data/email-Enron.txt.gz

Parsing completed. Parsed 367662 lines in 0.077463 secs.

In [28]:
print(g.summary())

{'num_edges': 367662, 'num_vertices': 36692}


<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 4
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Calculate PageRank for vertex 47.
    </div>
</div>

***

***
<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 5
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Calculate the number of triangles in the graph. 
    </div>
</div>  (A triangle is a complete subgraph with only three vertices.) 

***

***
<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 6
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Calculate the number of connected components in the graph. A connected component of an undirected graph is a subgraph in which any two vertices are connected to  each other by paths.
    </div>
</div>  

Reference 
https://apple.github.io/turicreate/docs/api/generated/turicreate.connected_components.ConnectedComponentsModel.html


In [32]:
from turicreate import connected_components
cc=connected_components.create(g)
print(cc.summary)

+-----------------------------+

| Number of components merged |

+-----------------------------+

| 114395                      |

| 0                           |

+-----------------------------+

<bound method CustomModel.summary of Class                                    : ConnectedComponentsModel

Graph
-----
num_edges                                : 367662
num_vertices                             : 36692

Results
-------
graph                                    : SGraph. See m['graph']
number of connected components           : 1065
component size                           : SFrame. See m['component_size']
vertex component id                      : SFrame. See m['component_id']

Metrics
-------
training time (secs)                     : 0.1096

Queryable Fields
----------------
component_id                             : An SFrame with each vertex's component id
component_size                           : An SFrame with the size of each component
graph                                    : A new SGraph with the color id as a vertex property
training_time                            : Total training time of the model
>


In [ ]:
## Place your answer Here

+-----------------------------+
| Number of components merged |
+-----------------------------+
| 114395                      |
| 0                           |
+-----------------------------+

***

***
<div style="width:100%;">
    <div style="width:15%;float:left;font-size:20px;background-color:#557aba;color:#eff3f9;padding:6px;font-wight:bold;text-align:center;">
    Question 7
    </div>
    <div style="width:85%;float:right;font-size:16px;background-color:#dce4f2;font-wight:normal;color:black;padding:6px;">
    Write component id for vertex 47.
    </div>
</div> 

In [33]:
cc_ids=cc['component_id']
print(cc_ids)

+------+--------------+
| __id | component_id |
+------+--------------+
|  5   |      18      |
|  7   |      18      |
|  8   |      18      |
|  10  |      18      |
|  27  |      18      |
|  29  |      18      |
|  33  |      18      |
|  47  |      18      |
|  50  |      18      |
|  52  |      18      |
+------+--------------+
[36692 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [ ]:
## Place your answer Here

__id  component_id
47    18

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>

Submission: Once you're finished, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can go to the URL that you got at the very beginning of this homework and flag the correct version. To do so, go to the website, click on this assignment, and find the version you would like to have graded. There should be an option to flag that submission for grading. Good luck!

In [ ]:
_ = ok.submit()

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>